# Simple Evolutionary Exploration  Walkthrough

This notebook contains instructions on how to use the SEE module, along with several examples. These instructions will cover the following parts: 
* [Import Image Files](#Import_Image_Files)
* [Manual Search](#Manual_Search)
* [Genetic Algorithm Search](#Genetic_Algorithm_Search)
* [Reading the Results](#Reading_the_Results)

----
<a name="Import_Image_Files"></a>

## Import Image Files

First import the following packages:

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline
import imageio

Next, read in the image to be segmented, and the ground truth segmentation mask of the image:

In [ ]:
img = imageio.imread('Image_data/Coco_2017_unlabeled//rgbd_plant/rgb_04_009_05.png')
gmask = imageio.imread('Image_data/Coco_2017_unlabeled/rgbd_new_label/label_04_009_05299.png')

The ground truth mask should only contain one channel, and should only contain as many unique values as there are segments. If these conditions are not met, consider relabeling the image, converting the image to grayscale, or indexing to only use one channel (see below).

In [ ]:
gmask = gmask[:, :, 0]

The following can now be used to see the imported images:

In [ ]:
plt.figure(figsize= (10, 5))
plt.subplot(121)
plt.imshow(img)
plt.title("Original Image")
plt.axis("off")

plt.subplot(122)
plt.imshow(gmask)
plt.title("Ground Truth Segmentation Mask")
plt.axis("off")

plt.tight_layout
plt.show()

----
<a name="Manual_Search"></a>

## Manual Search

First import image files, as well as the following packages:

In [ ]:
from see import JupyterGUI, Segmentors

Manual searching of parameters can easily be done using the provided GUI. Pre-established parameters can be put into the widget, or the parameter values can be changed using the sliders. To change the algorithm, simply change the `alg` input. For a list of available inputs print `Segmentors.algorithmspace`


In [ ]:
Segmentors.algorithmspace

In [ ]:
### Example of input for params
# params = ['FB', 7563, 0.13, 2060, 0.01, 4342, 850, 10, 0.57, 1863, 1543, 134, 3, 1, 0.35, (1, 1), 8.1, 'checkerboard', 'checkerboard', 3, 7625, -35, 0.0, 0.0, 0.0]
JupyterGUI.segmentwidget(img, gmask, params = None, alg = 'CT')

----
<a name="Genetic_Algorithm_Search"></a>

## Genetic Algorithm Search

First import image files, as well as the following packages:

In [ ]:
from see import GeneticSearch, Segmentors

To run the genetic algorithm, we need to initialize an instance of an evolver. The original image and ground truth segmentation image are inputs to it, along with an integer value for population size. This value sets how many indivudals are in our population. For this example, we'll set this number to be equal to 10.

In [ ]:
my_evolver = GeneticSearch.Evolver(img, gmask, pop_size=10)

Now that the evolver has been initialized, we can run the genetic algorithm for a specified number of generations (or iterations). Here we will set this number equal to 5.

In [ ]:
# warnings may appear when this runs
population = my_evolver.run(ngen=5)

----
<a name="Reading_the_Results"></a>

## Reading the Results

After the genetic algorithm is complete, we can retrieve the individuals that resulted in the lowest (best) fitness values by printing `my_evolver.hof`. These individuals are sorted according to fitness value, so to get the overal best individual, we can simply look at the first individual in the list. 

In [ ]:
print('Best Individual:\n', my_evolver.hof[0])

We can see the mask this individual generates by evaluating it, then plotting the result:

In [ ]:
seg = Segmentors.algoFromParams(my_evolver.hof[0])
mask = seg.evaluate(img)

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(img)
plt.title("Original Image")
plt.axis('off')

plt.subplot(122)
plt.imshow(mask)
plt.title("Segmentation")
plt.axis('off')

plt.tight_layout
plt.show()

We can also use `FitnessFunction` to calculate the final fitness value for this algorithm:

In [ ]:
print('Fitness Value: ', Segmentors.FitnessFunction(mask, gmask)[0])

If this value is satisfactory, we can then get usable code to run this algorithm anywhere, including outside this notebook. The `print_best_algorithm_code` function does this using the given individual:

In [ ]:
GeneticSearch.print_best_algorithm_code(my_evolver.hof[0])

With this code, make sure to import skimage, along with any input images this algorithm will be applied to.